## Стандартная реализация кода


In [2]:
import random

# Константы
NUM_BUSES = 8
ROUTE_DURATION = 70  # мин.
SHIFT_START = 6 * 60 + 5  # Начало смены в 6:05
SHIFT_END = (27 * 60)  # Конец смены в 3:00 (следующий день)
WORK_TIME_TK = 420  # 7 часов в минутах (без обеда)
LUNCH_BREAK_TK = 60  # обед в минутах
WORK_TIME_SHIFT = 660  # 11 часов в минутах (без учёта перерывов и последней смены)
MIN_BREAK_TIME = 15  # Минимальное время на перерыв
MAX_BREAK_TIME = 20  # Максимальное время на перерыв

class Driver:
    def __init__(self, driver_type):
        self.driver_type = driver_type  # 'TK' или 'сменный'
        self.schedule = []  # Расписание водителя
        self.total_work_time = 0
        self.lunch_schedule = []  # Расписание обедов для TK
        self.break_schedule = []  # Расписание перерывов для сменных водителей

    def add_route(self, start_time):
        self.schedule.append(start_time)
        self.total_work_time += ROUTE_DURATION

    def add_lunch(self, start_time):
        self.lunch_schedule.append(start_time)
        self.total_work_time += LUNCH_BREAK_TK  # Увеличиваем общее рабочее время на обед
    
    def add_break(self, start_time):
        self.break_schedule.append(start_time)
        self.total_work_time += random.randint(MIN_BREAK_TIME, MAX_BREAK_TIME)  # Увеличиваем общее рабочее время на перерыв

class Schedule:
    def __init__(self):
        self.drivers = [Driver('TK') for _ in range(NUM_BUSES // 2)] + \
                       [Driver('сменный') for _ in range(NUM_BUSES // 2)]
        random.shuffle(self.drivers)

    def get_random_start_time(self, driver):
        if driver.driver_type == 'TK':
            return random.randint(SHIFT_START, SHIFT_START + 120)  # случайное время в пределах первых двух часов
        return random.randint(SHIFT_START, SHIFT_END - WORK_TIME_SHIFT)  # случайное время для сменных водителей

    def create_weekday_schedule(self):
        for driver in self.drivers:
            if driver.driver_type == 'TK':
                self.schedule_tk(driver)
            else:
                self.schedule_shift(driver)

    def schedule_tk(self, driver):
        start_time = self.get_random_start_time(driver)

        # Заполнение расписания рейсов
        while start_time < SHIFT_END and driver.total_work_time < WORK_TIME_TK:
            driver.add_route(start_time)

            # Определение времени для обеда
            if driver.total_work_time >= 240 and len(driver.lunch_schedule) == 0:
                lunch_time = start_time + ROUTE_DURATION
                if lunch_time <= SHIFT_END - LUNCH_BREAK_TK:
                    driver.add_lunch(lunch_time)
                    start_time = lunch_time + LUNCH_BREAK_TK  # Обновление start_time после обеда
                    continue

            start_time += ROUTE_DURATION

        # Последний рейс до 27:00 (3:00)
        if driver.total_work_time < WORK_TIME_TK and start_time < SHIFT_END:
            driver.add_route(SHIFT_END - ROUTE_DURATION)

    def schedule_shift(self, driver):
        start_time = self.get_random_start_time(driver)
        work_cycles = 0  # Счетчик рабочих циклов

        while start_time < SHIFT_END and driver.total_work_time < WORK_TIME_SHIFT:
            driver.add_route(start_time)

            work_cycles += 1
            if work_cycles % 2 == 0 and driver.total_work_time < WORK_TIME_SHIFT:  # Перерыв каждые 2 часа работы
                break_time = start_time + ROUTE_DURATION
                if break_time + MIN_BREAK_TIME <= SHIFT_END:
                    driver.add_break(break_time)
                    start_time += random.randint(MIN_BREAK_TIME, MAX_BREAK_TIME)  # Случайный перерыв

            start_time += ROUTE_DURATION

        # Завершение последнего рейса
        if driver.total_work_time < WORK_TIME_SHIFT:
            driver.add_route(SHIFT_END - ROUTE_DURATION)

    def create_weekend_schedule(self):
        weekend_drivers = [Driver('TK') for _ in range(NUM_BUSES // 2)] + \
                          [Driver('сменный') for _ in range(NUM_BUSES // 2)]
        random.shuffle(weekend_drivers)

        for driver in weekend_drivers:
            start_time = self.get_random_start_time(driver)

            while start_time < SHIFT_END and driver.total_work_time < WORK_TIME_TK:
                driver.add_route(start_time)

                if driver.driver_type == 'TK':
                    if driver.total_work_time >= 240 and len(driver.lunch_schedule) == 0:
                        lunch_time = start_time + ROUTE_DURATION
                        if lunch_time <= SHIFT_END - LUNCH_BREAK_TK:
                            driver.add_lunch(lunch_time)
                            start_time = lunch_time + LUNCH_BREAK_TK
                            continue

                else:
                    work_cycles = driver.total_work_time // ROUTE_DURATION
                    if work_cycles > 0 and work_cycles % 2 == 0:
                        break_time = start_time + ROUTE_DURATION
                        if break_time + MIN_BREAK_TIME <= SHIFT_END:
                            driver.add_break(break_time)
                            start_time += random.randint(MIN_BREAK_TIME, MAX_BREAK_TIME)

                start_time += ROUTE_DURATION

        return weekend_drivers

# Генерация расписания для будней
weekday_schedule = Schedule()
weekday_schedule.create_weekday_schedule()

# Генерация расписания для выходных
weekend_drivers = weekday_schedule.create_weekend_schedule()

# Вывод расписания водителей первого типа (TK) на будние дни
print("\nРасписание водителей первого типа (TK) на будние дни:")
for idx, driver in enumerate(weekday_schedule.drivers):
    if driver.driver_type == 'TK':
        print(f"Водитель TK {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
        if driver.lunch_schedule:
            print(f"Обед: {', '.join(f'{lunch // 60:02}:{lunch % 60:02}' for lunch in driver.lunch_schedule)}")

# Вывод расписания для второго типа (сменный) на будние дни
print("\nРасписание водителей второго типа (сменный) на будние дни:")
for idx, driver in enumerate(weekday_schedule.drivers):
    if driver.driver_type == 'сменный':
        print(f"Водитель сменный {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
        print(f"Перерывы: {', '.join(f'{break_time // 60:02}:{break_time % 60:02}' for break_time in driver.break_schedule)}")

# Подсчет общего количества маршрутов и водителей на будние дни
total_routes_weekday = sum(len(driver.schedule) for driver in weekday_schedule.drivers)

# Вывод расписания на выходные
print("\nРасписание водителей на выходные:")
# Сначала выводим водителей TK
for idx, driver in enumerate(weekend_drivers):
    if driver.driver_type == 'TK':
        print(f"Водитель TK {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
        if driver.lunch_schedule:
            print(f"Обед: {', '.join(f'{lunch // 60:02}:{lunch % 60:02}' for lunch in driver.lunch_schedule)}")

# Затем выводим водителей сменного типа
for idx, driver in enumerate(weekday_schedule.drivers):
    if driver.driver_type == 'сменный':
        print(f"Водитель сменный {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
        print(f"Перерывы: {', '.join(f'{break_time // 60:02}:{break_time % 60:02}' for break_time in driver.break_schedule)}")

# Подсчет общего количества маршрутов и водителей на выходные
total_routes_weekend = sum(len(driver.schedule) for driver in weekend_drivers)

# Генерация расписания автобусов для будних дней
buses_schedule_weekday = {f'Автобус {i + 1}': [] for i in range(NUM_BUSES)}

# Заполнение расписания автобусов на будние дни
for driver in weekday_schedule.drivers:
    for route_start_time in driver.schedule:
        bus_index = (driver.driver_type == 'TK') * (NUM_BUSES // 2) + (weekday_schedule.drivers.index(driver) % (NUM_BUSES // 2))
        buses_schedule_weekday[f'Автобус {bus_index + 1}'].append(route_start_time)

# Генерация расписания автобусов для выходных
buses_schedule_weekend = {f'Автобус {i + 1}': [] for i in range(NUM_BUSES)}

# Заполнение расписания автобусов на выходные
for driver in weekend_drivers:
    for route_start_time in driver.schedule:
        bus_index = (driver.driver_type == 'TK') * (NUM_BUSES // 2) + (weekend_drivers.index(driver) % (NUM_BUSES // 2))
        buses_schedule_weekend[f'Автобус {bus_index + 1}'].append(route_start_time)

# Объединение расписания автобусов при наличии "дыр" для будней
def merge_buses_schedule(buses_schedule):
    merged_buses_schedule = {}
    for bus, times in buses_schedule.items():
        merged_times = []
        last_end_time = None
        for time in sorted(times):
            if last_end_time is None or time - last_end_time > 0:
                merged_times.append(time)
            last_end_time = time + ROUTE_DURATION  # Время следующего рейса
        merged_buses_schedule[bus] = merged_times
    return merged_buses_schedule

# Полное заполнение расписания для автобусов
def complete_buses_schedule(merged_buses_schedule):
    for bus in merged_buses_schedule.keys():
        last_time = merged_buses_schedule[bus][-1] if merged_buses_schedule[bus] else SHIFT_START
        while len(merged_buses_schedule[bus]) < 24:  # Предполагаем, что максимум 24 рейса (с 6:05 до 3:00)
            next_time = last_time + 10  # Добавляем интервал 10 минут для следующего рейса
            if next_time < SHIFT_END:
                merged_buses_schedule[bus].append(next_time)
                last_time = next_time
            else:
                break  # Если следующий рейс выходит за пределы смены, выходим из цикла
    return merged_buses_schedule

# Объединение и заполнение расписания для будней
merged_buses_schedule_weekday = merge_buses_schedule(buses_schedule_weekday)
completed_buses_schedule_weekday = complete_buses_schedule(merged_buses_schedule_weekday)

# Вывод расписания автобусов на будние дни
print("\nРасписание автобусов на будние дни:")
for bus, schedule in completed_buses_schedule_weekday.items():
    print(f"{bus}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in schedule)}")

# Объединение и заполнение расписания для выходных
merged_buses_schedule_weekend = merge_buses_schedule(buses_schedule_weekend)
completed_buses_schedule_weekend = complete_buses_schedule(merged_buses_schedule_weekend)

# Вывод расписания автобусов на выходные
print("\nРасписание автобусов на выходные:")
for bus, schedule in completed_buses_schedule_weekend.items():
    print(f"{bus}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in schedule)}")

# Учет работы водителей первого и второго типа
total_drivers_required = len(weekday_schedule.drivers)
total_drivers_tk = total_routes_weekday // 5  # 5 дней в неделю
total_drivers_shift = (total_routes_weekday // WORK_TIME_SHIFT) * 2  # Работают через 2 дня

print(f"\nОбщее количество маршрутов в будние дни: {total_routes_weekday}")
print(f"Общее количество водителей, необходимых для будних дней: {total_drivers_tk + total_drivers_shift}")

# Подсчет общего количества маршрутов и водителей на выходные
total_drivers_weekend = len(weekend_drivers)
print(f"Общее количество маршрутов на выходные: {total_routes_weekend}")
print(f"Общее количество водителей, необходимых для выходных: {total_drivers_weekend}")



Расписание водителей первого типа (TK) на будние дни:
Водитель TK 3: 07:24, 08:34, 09:44, 10:54, 13:04, 14:14
Обед: 12:04
Водитель TK 5: 07:51, 09:01, 10:11, 11:21, 13:31, 14:41
Обед: 12:31
Водитель TK 6: 07:06, 08:16, 09:26, 10:36, 12:46, 13:56
Обед: 11:46
Водитель TK 7: 07:11, 08:21, 09:31, 10:41, 12:51, 14:01
Обед: 11:51

Расписание водителей второго типа (сменный) на будние дни:
Водитель сменный 1: 06:17, 07:27, 08:52, 10:02, 11:32, 12:42, 14:10, 15:20, 16:45
Перерывы: 08:37, 11:12, 13:52, 16:30
Водитель сменный 2: 07:02, 08:12, 09:41, 10:51, 12:16, 13:26, 14:52, 16:02, 17:27
Перерывы: 09:22, 12:01, 14:36, 17:12
Водитель сменный 4: 15:41, 16:51, 18:20, 19:30, 20:57, 22:07, 23:37, 24:47, 26:13
Перерывы: 18:01, 20:40, 23:17, 25:57
Водитель сменный 8: 07:31, 08:41, 10:10, 11:20, 12:48, 13:58, 15:25, 16:35, 18:05
Перерывы: 09:51, 12:30, 15:08, 17:45

Расписание водителей на выходные:
Водитель TK 3: 07:48, 08:58, 10:08, 11:18, 13:28, 14:38
Обед: 12:28
Водитель TK 6: 07:15, 08:25, 09:35

## Генетический алгоритм будет включать в себя следующие шаги:
1. Инициализация популяции: Создание начального набора расписаний.
2. Оценка приспособленности: Оценка каждого расписания по критериям, таким как количество необходимых водителей и количество маршрутов.
3. Селекция: Выбор лучших расписаний для создания следующего поколения.
4. Скрещивание и мутация: Генерация новых расписаний на основе выбранных.
5. Повторение: Повторение шагов 2-4 до достижения условия остановки.

In [4]:
import random

# Константы
NUM_BUSES = 8
ROUTE_DURATION = 70  # мин.
SHIFT_START = 6 * 60 + 5  # Начало смены в 6:05
SHIFT_END = (27 * 60)  # Конец смены в 3:00 (следующий день)
WORK_TIME_TK = 360  # 6 часов в минутах (без обеда)
LUNCH_BREAK_TK = 60  # обед в минутах
WORK_TIME_SHIFT = 660  # 11 часов в минутах (без учета перерывов)
MIN_BREAK_TIME = 15  # Минимальное время на перерыв
MAX_BREAK_TIME = 20  # Максимальное время на перерыв
MUTATION_RATE = 0.1  # Вероятность мутации
POPULATION_SIZE = 50  # Размер популяции
GENERATIONS = 100  # Количество генераций

class Driver:
    def __init__(self, driver_type):
        self.driver_type = driver_type  # 'TK' или 'сменный'
        self.schedule = []  # Расписание водителя
        self.total_work_time = 0
        self.lunch_schedule = []  # Расписание обедов для TK
        self.break_schedule = []  # Расписание перерывов для сменных водителей

    def add_route(self, start_time):
        self.schedule.append(start_time)
        self.total_work_time += ROUTE_DURATION

    def add_lunch(self, start_time):
        self.lunch_schedule.append(start_time)
        self.total_work_time += LUNCH_BREAK_TK

    def add_break(self, start_time):
        self.break_schedule.append(start_time)
        self.total_work_time += random.randint(MIN_BREAK_TIME, MAX_BREAK_TIME)

class Schedule:
    def __init__(self):
        self.drivers = [Driver('TK') for _ in range(NUM_BUSES // 2)] + \
                       [Driver('сменный') for _ in range(NUM_BUSES // 2)]
        random.shuffle(self.drivers)
        self.create_schedule()

    def get_random_start_time(self, driver):
        if driver.driver_type == 'TK':
            return random.randint(SHIFT_START, SHIFT_START + 120)  # случайное время в пределах первых двух часов
        return random.randint(SHIFT_START, SHIFT_END - WORK_TIME_SHIFT)  # случайное время для сменных водителей

    def create_schedule(self):
        for driver in self.drivers:
            if driver.driver_type == 'TK':
                self.schedule_tk(driver)
            else:
                self.schedule_shift(driver)

    def schedule_tk(self, driver):
        start_time = self.get_random_start_time(driver)
        while start_time < SHIFT_END and driver.total_work_time < WORK_TIME_TK:
            driver.add_route(start_time)
            if driver.total_work_time >= 240 and len(driver.lunch_schedule) == 0:
                lunch_time = start_time + ROUTE_DURATION
                if lunch_time <= SHIFT_END - LUNCH_BREAK_TK:
                    driver.add_lunch(lunch_time)
                    start_time = lunch_time + LUNCH_BREAK_TK
            start_time += ROUTE_DURATION

        if driver.total_work_time < WORK_TIME_TK and start_time < SHIFT_END:
            driver.add_route(SHIFT_END - ROUTE_DURATION)

    def schedule_shift(self, driver):
        start_time = self.get_random_start_time(driver)
        work_cycles = 0
        while start_time < SHIFT_END and driver.total_work_time < WORK_TIME_SHIFT:
            driver.add_route(start_time)
            work_cycles += 1
            if work_cycles % 2 == 0 and driver.total_work_time < WORK_TIME_SHIFT:  # Перерыв каждые 2 часа работы
                break_time = start_time + ROUTE_DURATION
                if break_time + MIN_BREAK_TIME <= SHIFT_END:
                    driver.add_break(break_time)
                    start_time += random.randint(MIN_BREAK_TIME, MAX_BREAK_TIME)
            start_time += ROUTE_DURATION

        if driver.total_work_time < WORK_TIME_SHIFT:
            driver.add_route(SHIFT_END - ROUTE_DURATION)

    def fitness(self):
        return sum(driver.total_work_time for driver in self.drivers)  # Чем меньше, тем лучше

    def calculate_route_count(self):
        return sum(len(driver.schedule) for driver in self.drivers)  # Общее количество маршрутов

    def calculate_driver_count(self):
        return len(self.drivers)  # Общее количество водителей

class Population:
    def __init__(self, size):
        self.schedules = [Schedule() for _ in range(size)]

    def evaluate(self):
        for schedule in self.schedules:
            schedule.fitness()

    def selection(self):
        # Сортируем расписания по фитнес-значениям и выбираем лучшие
        self.schedules.sort(key=lambda x: x.fitness())
        return self.schedules[:POPULATION_SIZE // 2]  # Выбираем половину лучших

    def crossover(self, parent1, parent2):
        child1 = Schedule()
        child2 = Schedule()

        # Кроссовер водителей между двумя родителями
        half = len(parent1.drivers) // 2
        child1.drivers = parent1.drivers[:half] + parent2.drivers[half:]
        child2.drivers = parent2.drivers[:half] + parent1.drivers[half:]

        return child1, child2

    def mutate(self, schedule):
        # Случайно изменяем расписание одного водителя
        random_driver = random.choice(schedule.drivers)
        random_start_time = random.randint(SHIFT_START, SHIFT_START + 120)  # Новое случайное время
        random_driver.schedule = []  # Очистка текущего расписания
        random_driver.total_work_time = 0
        random_driver.lunch_schedule = []
        random_driver.break_schedule = []
        if random_driver.driver_type == 'TK':
            schedule.schedule_tk(random_driver)
        else:
            schedule.schedule_shift(random_driver)

# Генерация расписания для выходных
def generate_weekend_schedule():
    return Schedule()

weekend_schedule = generate_weekend_schedule()

# Функция для вывода расписания водителей
def print_driver_schedules(schedules, title):
    print(f"\n{title}:")
    # Сначала водители TK
    for idx, driver in enumerate(schedules.drivers):
        if driver.driver_type == 'TK':
            print(f"Водитель TK {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
            if driver.lunch_schedule:
                print(f"    Обед: {', '.join(f'{lunch // 60:02}:{lunch % 60:02}' for lunch in driver.lunch_schedule)}")
    # Затем сменные водители
    for idx, driver in enumerate(schedules.drivers):
        if driver.driver_type == 'сменный':
            print(f"Водитель сменный {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
            print(f"    Перерывы: {', '.join(f'{break_time // 60:02}:{break_time % 60:02}' for break_time in driver.break_schedule)}")

# Функция для вывода расписания автобусов
def print_bus_schedule(schedules, title):
    print(f"\n{title}:")
    bus_schedule = {i: [] for i in range(NUM_BUSES)}

    for sch in schedules:
        for idx in range(NUM_BUSES):
            times = [start_time for start_time in sch.drivers[idx].schedule]
            bus_schedule[idx].extend(times)

    # Сортировка и фильтрация уникальных времен
    for idx in bus_schedule:
        bus_schedule[idx] = sorted(set(bus_schedule[idx]))

    # Вывод расписания в порядке возрастания времени
    for idx in bus_schedule:
        times = ', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in bus_schedule[idx])
        print(f"Автобус {idx + 1}: {times}")

# Функции для вывода общих расчетов
def print_summary(schedules_weekday, weekend_schedule):
    total_routes_weekday = sum(schedule.calculate_route_count() for schedule in schedules_weekday)
    total_drivers_weekday = len(schedules_weekday[0].drivers)  # Общее количество водителей в расписаниях
    total_routes_weekend = weekend_schedule.calculate_route_count()
    total_drivers_weekend = len(weekend_schedule.drivers)  # Общее количество водителей в выходные

    print("\nОбщие расчёты:")
    print(f"Общее количество маршрутов в будние дни: {total_routes_weekday}")
    print(f"Общее количество водителей, необходимых для будних дней: {total_drivers_weekday}")
    print(f"Общее количество маршрутов в выходные дни: {total_routes_weekend}")
    print(f"Общее количество водителей, необходимых для выходных дней: {total_drivers_weekend}")

# Дополнительные функции для печати расписаний отдельно
def print_weekend_schedules(schedule):
    print("\nРасписание водителей в выходные дни:")
    # Сначала водители TK
    for idx, driver in enumerate(schedule.drivers):
        if driver.driver_type == 'TK':
            print(f"Водитель TK {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
            if driver.lunch_schedule:
                print(f"    Обед: {', '.join(f'{lunch // 60:02}:{lunch % 60:02}' for lunch in driver.lunch_schedule)}")
    # Затем сменные водители
    for idx, driver in enumerate(schedule.drivers):
        if driver.driver_type == 'сменный':
            print(f"Водитель сменный {idx + 1}: {', '.join(f'{start_time // 60:02}:{start_time % 60:02}' for start_time in driver.schedule)}")
            print(f"    Перерывы: {', '.join(f'{break_time // 60:02}:{break_time % 60:02}' for break_time in driver.break_schedule)}")

# Запуск алгоритма
population = Population(POPULATION_SIZE)

# Генерация трех расписаний для будней
schedules_weekday = []
for _ in range(3):
    population.evaluate()
    best_schedule = min(population.schedules, key=lambda x: x.fitness())
    schedules_weekday.append(best_schedule)

# Вывод расписаний в нужном порядке
print_driver_schedules(schedules_weekday[0], "Расписание водителей в будние дни")
print_weekend_schedules(weekend_schedule)
print_bus_schedule(schedules_weekday, "Расписание автобусов в будние дни")
print_bus_schedule([weekend_schedule], "Расписание автобусов в выходные дни")

# Печать общих расчетов
print_summary(schedules_weekday, weekend_schedule)



Расписание водителей в будние дни:
Водитель TK 5: 06:32, 07:42, 08:52, 10:02, 13:22
    Обед: 11:12
Водитель TK 6: 06:34, 07:44, 08:54, 10:04, 13:24
    Обед: 11:14
Водитель TK 7: 06:35, 07:45, 08:55, 10:05, 13:25
    Обед: 11:15
Водитель TK 8: 06:38, 07:48, 08:58, 10:08, 13:28
    Обед: 11:18
Водитель сменный 1: 08:39, 09:49, 11:15, 12:25, 13:52, 15:02, 16:27, 17:37, 19:02
    Перерывы: 10:59, 13:35, 16:12, 18:47
Водитель сменный 2: 08:55, 10:05, 11:35, 12:45, 14:12, 15:22, 16:52, 18:02, 19:27
    Перерывы: 11:15, 13:55, 16:32, 19:12
Водитель сменный 3: 13:24, 14:34, 16:03, 17:13, 18:41, 19:51, 21:17, 22:27, 23:55
    Перерывы: 15:44, 18:23, 21:01, 23:37
Водитель сменный 4: 13:00, 14:10, 15:39, 16:49, 18:16, 19:26, 20:51, 22:01, 23:26
    Перерывы: 15:20, 17:59, 20:36, 23:11

Расписание водителей в выходные дни:
Водитель TK 2: 06:24, 07:34, 08:44, 09:54, 13:14
    Обед: 11:04
Водитель TK 3: 07:05, 08:15, 09:25, 10:35, 13:55
    Обед: 11:45
Водитель TK 4: 08:02, 09:12, 10:22, 11:32, 1